## Electromagnetic wave animation ##

We animate the waves $E_z=A\sin(y/\lambda-\omega t), x=0$, respectively $E_x=A\sin(y/\lambda-\omega t), z=0$, that propagate in the positive Oy direction. $\lambda$ is the wave length, and $\omega$ is the angular frequency.

More precisely, we plot at each t in a grid of the interval of simulation,  the electric field and the magnetic field of the electromagnetic wave.

In [1]:
import numpy as np

Define a function that returns data to draw  vectors of a vector field with values in the plane yOz or xOy (i.e. vectors of the electric or magnetic field, in our case). 

In [7]:
def rot_matrix(theta):
    return np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])

def get_arrows(start, end, arrow_angle,  plane=2, fract=0.1):# this is  function for defining vectors/quivers
    #start is the numpy array of x,y, z-coordinates of the starting points of arrows; shape (3, m)
    #start[0,:] contains x-coordinates, etc
    #end is a numpy array with the same shape as start; contains on rows the x,  y and z-coords 
    # of ending points of the arrow  
    #the arrowhead is an isosceles triangle with the equal sides forming an angle of 2*arrow_angle radians 
    #plane=0 or 2 depending on the plane where the vectors are drawn (plane=0 i.e. x=0, plane=2, z=0)
    
    start=np.asarray(start)
    end=np.asarray(end)
    m=start[0,:].shape[0]
    arr_dir=start-end
    arr_dir_norm=np.linalg.norm(arr_dir, axis=0)
    arr_dir=fract*arr_dir/arr_dir_norm[None,:]#the arrowhead is a fraction fract from the unit vector
    if plane==2:# i.e. we plot vectors in z=0
        v=np.einsum('ji, im -> jm', rot_matrix(arrow_angle), arr_dir[:plane,:])#Einstein summation to apply                                                                                  # rotation to all vectors at a time
        w=np.einsum('ji, im -> jm', rot_matrix(-arrow_angle), arr_dir[:plane, :])
        v=np.append(v, [[0]*m], axis=0) 
        w=np.append(w, [[0]*m], axis=0) 
    elif plane==0:# plot vectors in x=0
        v=np.einsum('ji, im -> jm',  rot_matrix(arrow_angle), arr_dir[1:,:])                         
        w=np.einsum('ji, im -> jm', rot_matrix(-arrow_angle), arr_dir[1:, :])
        v=np.append([[0]*m], v, axis=0)
        w=np.append([[0]*m], w, axis=0)
    else: raise ValueError('the key plane must be 0 or 2')    
    p=end+v
    q=end+w

    suppx=np.stack((start[0,:], end[0,:], np.nan*np.ones(m )))#supp is the line segment as support for arrow 
    suppy=np.stack((start[1,:], end[1,:], np.nan*np.ones(m )))
    suppz=np.stack((start[2,:], end[2,:], np.nan*np.ones(m )))
    x=suppx.flatten('F')#Fortran type flattening
    y=suppy.flatten('F')
    z=suppz.flatten('F')
    x=map(lambda u: None if np.isnan(u) else u, x)
    y=map(lambda u: None if np.isnan(u) else u, y)
    z=map(lambda u: None if np.isnan(u) else u, z)
    
    #headx, heady, headz the x, y, z coordinates of the triangle vertices
    headx=np.stack((end[0,:], p[0,:], q[0,:], end[0,:], np.nan*np.ones(m)))
    heady=np.stack((end[1,:], p[1,:], q[1,:], end[1,:], np.nan*np.ones(m)))
    headz=np.stack((end[2,:], p[2,:], q[2,:], end[2,:], np.nan*np.ones(m)))               
    xx=headx.flatten('F')
    yy=heady.flatten('F')
    zz=headz.flatten('F')               
    xx=map(lambda u: None if np.isnan(u) else u, xx)
    yy=map(lambda u: None if np.isnan(u) else u, yy)
    zz=map(lambda u: None if np.isnan(u) else u, zz)               
    
    return x, y, z, xx, yy, zz
    

In [8]:
import plotly.plotly as py
from plotly.grid_objs import Grid, Column

In [9]:
py.sign_in('empet', 'api-key')

Define data for fixed traces in each frame, representing the two orthogonal planes of the electric and magnetic field:

In [15]:
a=2
b=5
#data to 
xblue=[-a, a, a , -a, -a]
yblue=[-b, -b, b, b, -b]
zblue=[0]*5
my_columns=[Column(xblue, 'xb'),Column(yblue, 'yb'), Column(zblue, 'zb')]

xred=[0]*5+[None, 0, 0, 0]
yred=[-b, -b, b, b, -b, None, -b, b]
zred=[a, -a, -a, a, a, None, 0, 0]
my_columns+=[Column(xred, 'xr'),Column(yred, 'yr'), Column(zred, 'zr')]

x_Oy=[0,0]
y_Oy=[-b,b]
z_Oy=[0,0]
my_columns+=[Column(x_Oy, 'xy'), Column(y_Oy, 'yy'), Column(z_Oy, 'zy')]


Set the wave parameters and the interval of simulation:

In [16]:
A=1# wave amplitude
lambdA=0.5#wave length
omega=1# angular frequency
t=np.arange(0., 10., 0.2)# interval of simulation
Y=np.arange(-b, b, 0.2)
X=np.zeros(Y.shape[0])
ZZe=np.zeros(Y.shape[0])

In [17]:
nr_frames=t.shape[0]
theta=np.pi/13# the characteristic angle of each arrow
start=np.stack((X, Y, np.zeros(X.shape)))# the numpy array of strating points of the the two classes of vectors
start.shape

(3L, 50L)

Define data representing the vectors of the two vector fields:

In [18]:
for k in range(nr_frames):
    Ze=A*np.sin(Y/lambdA-omega*t[k])
    end1=np.stack((X, Y, Ze))
    x1,y1,z1, xx1, yy1, zz1=get_arrows(start, end1, theta, plane=0)
    my_columns+=[Column(x1, 'x1_{}'.format(k+1)), Column(y1, 'y1_{}'.format(k+1)), Column(z1, 'z1_{}'.format(k+1))]
    my_columns+=[Column(xx1, 'xx1_{}'.format(k+1)), Column(yy1, 'yy1_{}'.format(k+1)), Column(zz1, 'zz1_{}'.format(k+1))]
    XXe=A*np.sin(Y/lambdA-omega*t[k])
    end2=np.stack((XXe, Y, ZZe))
    x2,y2,z2, xx2, yy2, zz2=get_arrows(start, end2, theta, plane=2)
    my_columns+=[Column(x2, 'x2_{}'.format(k+1)), Column(y2, 'y2_{}'.format(k+1)), Column(z2, 'z2_{}'.format(k+1))]
    my_columns+=[Column(xx2, 'xx2_{}'.format(k+1)), Column(yy2, 'yy2_{}'.format(k+1)), Column(zz2, 'zz2_{}'.format(k+1))]
    
grid = Grid(my_columns)
py.grid_ops.upload(grid, 'electromg-wave1', auto_open=False)  

u'https://plot.ly/~empet/14296/'

The following four traces are the base traces updated by the animation frames:

In [19]:
tr0=dict(type='scatter3d',  # a rectangle in xOy  
         xsrc=grid.get_column_reference('xb'),
         ysrc=grid.get_column_reference('yb'),
         zsrc=grid.get_column_reference('zb'), 
         mode='lines',
         line=dict(width=1.5, color='blue')  
       )
tr1=dict(type='scatter3d',# a rectangle in yOz
         xsrc=grid.get_column_reference('xr'),
         ysrc=grid.get_column_reference('yr'),
         zsrc=grid.get_column_reference('zr'), 
         mode='lines',
         line=dict(width=1.5, color='red')
        )
tr2=dict(type='scatter3d',#line of direction Oy
         xsrc=grid.get_column_reference('xy'),
         ysrc=grid.get_column_reference('yy'),
         zsrc=grid.get_column_reference('zy'), 
         mode='lines',
         line=dict(width=1.5, color='rgb(140,140,140)')
        )
        

In [21]:
tr3=dict(type='scatter3d',
         xsrc=grid.get_column_reference('x1_1'),
         ysrc=grid.get_column_reference('y1_1'),
         zsrc=grid.get_column_reference('z1_1'),
         mode='lines', 
         line=dict(color='red', width=1.5),
         name=''
        )
                                   
tr4=dict(type='scatter3d',
         xsrc=grid.get_column_reference('xx1_1'),
         ysrc=grid.get_column_reference('yy1_1'),
         zsrc=grid.get_column_reference('zz1_1'),      
         mode='lines', 
         line=dict(color='red', width=2), 
         name='' 
        )
tr5=dict(type='scatter3d',
         xsrc=grid.get_column_reference('x2_1'),
         ysrc=grid.get_column_reference('y2_1'),
         zsrc=grid.get_column_reference('z2_1'),
         mode='lines', 
         line=dict(color='blue', width=1.5),
         name=''
        )
tr6=dict(type='scatter3d',
         xsrc=grid.get_column_reference('xx2_1'),
         ysrc=grid.get_column_reference('yy2_1'),
         zsrc=grid.get_column_reference('zz2_1'),  
         mode='lines', 
         line=dict(color='blue', width=2), 
         name=''
        )

In [24]:
frames=[]
for k in range(nr_frames):
    frames+=[dict(data=[dict(xsrc=grid.get_column_reference('x1_{}'.format(k+1)),
                             ysrc=grid.get_column_reference('y1_{}'.format(k+1)),
                             zsrc=grid.get_column_reference('z1_{}'.format(k+1))
                           ),
                        dict(xsrc=grid.get_column_reference('xx1_{}'.format(k+1)),
                             ysrc=grid.get_column_reference('yy1_{}'.format(k+1)),
                             zsrc=grid.get_column_reference('zz1_{}'.format(k+1))
                            ),
                       dict(xsrc=grid.get_column_reference('x2_{}'.format(k+1)),
                            ysrc=grid.get_column_reference('y2_{}'.format(k+1)),
                            zsrc=grid.get_column_reference('z2_{}'.format(k+1))
                           ),
                       dict(xsrc=grid.get_column_reference('xx2_{}'.format(k+1)),
                            ysrc=grid.get_column_reference('yy2_{}'.format(k+1)),
                            zsrc=grid.get_column_reference('zz2_{}'.format(k+1))
                            )
                       ],
                  traces=[3,4,5,6]
                 )
           ]

In [25]:
data=[tr0, tr1, tr2, tr3, tr4, tr5, tr6]

Set the plot layout:

In [26]:
noaxis=dict( showbackground=False,
            showgrid=False,
            showline=False,
            showticklabels=False,
            ticks='',
            title='',
            zeroline=False)
title='Electromagnetic wave propagating in the positive Oy direction<br>'+\
'The  electric field vectors (red) are included in the plane yOz,<br> and  the magnetic field vectors (blue), in xOy'
layout = dict(title=title,
              font=dict(family='Balto'),
              autosize=False,
              width=700,
              height=700,
              showlegend=False,
              scene=dict(camera = dict(eye=dict(x=1.22, y=0.55, z=0.3)),
              aspectratio=dict(x=1, y=1, z=0.65),
              xaxis=dict(noaxis),
              yaxis=dict(noaxis), 
              zaxis=dict(noaxis),
               
                        ),
               updatemenus=[dict(type='buttons', showactive=False,
                                y=0.75,
                                x=1.05,
                                xanchor='left',
                                yanchor='top',
                                pad=dict(t=0, l=10),
                                buttons=[dict(label='Play',
                                              method='animate',
                                              args=[None, 
                                                    dict(frame=dict(duration=80, 
                                                                    redraw=False),
                                                         transition=dict(duration=0),
                                                         fromcurrent=True,
                                                         mode='immediate'
                                                        )
                                                   ]
                                             )
                                        ]
                               )
                          ]     
    
)


In [27]:
fig=dict(data=data, layout=layout, frames=frames)
py.icreate_animations(fig, filename='anim-electromagwave')


In [28]:
from IPython.core.display import HTML
def  css_styling():
    styles = open("./custom.css", "r").read()
    return HTML(styles)
css_styling()